# Get simulation results. 

This notebook automatically collect and plot accuracy vs numlabels graphs for best and latest models for `efficientnet-lite0` and `unet` networks. Results are saved as `PNG` images.

## 1) Imports.

In [ ]:
# Utility imports
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import os
from glob import glob
import sys
sys.path.append("..")

# Main imports
import torch
import MSMatch as mm
from MSMatch.utils.load_cfg import load_cfg
from termcolor import colored
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
simulation_path="./saved_models/test"
dataset="eurosat_rgb"
cfg_path="../cfgs"

## 2) Getting configurations

In [ ]:
print("Parsing simulation dir: "+colored(os.path.join(simulation_path,dataset), "red"))
simulation_dirs=sorted(glob(os.path.join(simulation_path,dataset, "*")))
print("Number of tests found: ", colored(len(simulation_dirs), "red"))
cfg_config=[]
cfg_names=[]
for path in simulation_dirs:
    numlabels=int(path[path.find("numlabels")+9:path.find("numlabels")+9+path[path.find("numlabels")+9:].find("_")])
    net=path[path.find("arch")+4:path.find("arch")+4+path[path.find("arch")+4:].find("_")]
    cfg_config.append([numlabels, net,os.path.join(cfg_path, "cfg_"+net+"_"+str(numlabels)+".toml")])
    cfg_names.append("cfg_"+net+"_"+str(numlabels)+".toml")
    
cfg_dict=dict(zip(cfg_names, cfg_config))


## 3) Loading and testing trained models

In [ ]:
# Creating empty dictionaries for best and latest models
simulation_points=[50,100,200]
unet_best=dict(zip(simulation_points, [0 for n in simulation_points]))
unet_latest=dict(zip(simulation_points, [0 for n in simulation_points]))
efficientnet_best=dict(zip(simulation_points, [0 for n in simulation_points]))
efficientnet_latest=dict(zip(simulation_points, [0 for n in simulation_points]))


# This variable is used to avoid to load the dataset if it has the same seed of the previous run.
last_seed=-1


#Testing trained models
for cfg_name, n in zip(cfg_names, range(len(cfg_names))):

    print(colored("\n--- Inspecting: "+simulation_dirs[n]+" ---", "blue"))

    #Loading dataset
    cfg=load_cfg(cfg_dict[cfg_name][2])
    
    if last_seed != cfg.seed:
        print("Loading dataset: "+colored(dataset,"green"))
        _eval_dset = mm.SSL_Dataset(name=dataset, train=False, data_dir=None, seed=cfg.seed,)
        eval_dset = _eval_dset.get_dset()
        eval_loader = mm.get_data_loader(eval_dset, 32, num_workers=1)
        last_seed=cfg.seed

    cfg.num_classes = _eval_dset.num_classes
    cfg.num_channels = _eval_dset.num_channels
    
    
    print("Loading model: "+colored(cfg_dict[cfg_name][1],"red"))
    net_builder = mm.get_net_builder(cfg.net,pretrained=cfg.pretrained, in_channels=cfg.num_channels, scale=cfg.scale)
    net = net_builder(num_classes=cfg.num_classes, in_channels=cfg.num_channels)
    
    for model_type in ["latest_model.pth", "model_best.pth"]:
        print("Model type: "+colored(model_type, "cyan"))
        net.load_state_dict(torch.load(os.path.join(simulation_dirs[n], model_type))["eval_model"])
        if torch.cuda.is_available():
            net.cuda()
        net.eval() 
    
        acc = 0.0
        with torch.no_grad():
            for image, target in eval_loader:
                if torch.cuda.is_available():
                    image = image.type(torch.FloatTensor).cuda()
                logit = net(image)
    
                acc += logit.cpu().max(1)[1].eq(target).sum().numpy()
    
        acc=acc/len(eval_dset)
        print(f"Test Accuracy: {acc}")
        
        if cfg_dict[cfg_name][1] == "efficientnet-lite0":
            if model_type == "latest_model.pth":
                efficientnet_latest[cfg_dict[cfg_name][0]]=acc * 100
            else:
                efficientnet_best[cfg_dict[cfg_name][0]]=acc * 100
        else:
            if model_type == "latest_model.pth":
                unet_best[cfg_dict[cfg_name][0]]=acc * 100
            else:
                unet_latest[cfg_dict[cfg_name][0]]=acc * 100

## 4) Plotting models

### Best models

In [ ]:
fig_best=plt.figure(figsize=(10,10))

In [ ]:
# Getting values from dictionary.
efficientnet_best_values=[efficientnet_best[x] for x in efficientnet_best.keys()]
unet_best_values=[unet_best[x] for x in unet_best.keys()]

f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)

plt_ef = plt.plot(simulation_points, efficientnet_best_values, linestyle=":")
stem_ef = plt.plot(simulation_points, efficientnet_best_values, 'ro')
plt_unet= plt.plot(simulation_points, unet_best_values, linestyle=":")
stem_unet = plt.plot(simulation_points, unet_best_values, 'k*')

 
for (x,n) in zip(efficientnet_best_values, range(len(efficientnet_best_values))): 
    if n==0:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x+0.55))
    elif n == len(efficientnet_best_values) - 1:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n]-8, x-0.55))
    else:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x-0.55))

for (x,n) in zip(unet_best_values, range(len(unet_best_values))): 
    if n==0:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x+0.55))
    elif n == len(unet_best_values) - 1:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n]-8, x-0.55))
    else:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x-0.55))

plt.xlabel("numlabels")
plt.ylabel('accuracy [%]')
plt.legend(["efficientnet_lite0", "efficientnet_lite0", "unet", 'unet'])
plt.title("Best models: Accuracy vs Numlabels")
plt.savefig("best.png")

### Latest models

In [ ]:
# Getting values from dictionary.
efficientnet_latest_values=[efficientnet_latest[x] for x in efficientnet_latest.keys()]
unet_latest_values=[unet_latest[x] for x in unet_latest.keys()]

plt.close()
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)
plt_ef = plt.plot(simulation_points, efficientnet_latest_values, linestyle=":")
stem_ef = plt.plot(simulation_points, efficientnet_latest_values, 'ro')
plt_unet= plt.plot(simulation_points, unet_latest_values, linestyle=":")
stem_unet = plt.plot(simulation_points, unet_latest_values, 'k*')


for (x,n) in zip(efficientnet_latest_values, range(len(efficientnet_latest_values))): 
    if n==0:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x+0.55))
    elif n == len(efficientnet_latest_values) - 1:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n]-8, x-0.55))
    else:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x-0.55))

for (x,n) in zip(unet_latest_values, range(len(unet_latest_values))): 
    if n==0:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x+0.55))
    elif n == len(unet_latest_values) - 1:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n]-8, x-0.55))
    else:
        plt.annotate("{:.2f}".format(x), xy=(simulation_points[n], x), xytext=(simulation_points[n], x-0.55))

plt.xlabel("numlabels")
plt.ylabel('accuracy [%]')
plt.legend(["efficientnet_lite0", "efficientnet_lite0", "unet", 'unet'])
plt.title("Latest models: Accuracy vs Numlabels")
plt.savefig("latest.png")